In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import eugene as eu

# ProfileDataset

In [5]:
import numpy as np
import torch
import pandas as pd

import pyfaidx
import pyBigWig

from tqdm import tqdm

def extract_loci(
	loci, 
    sequences, 
    signals=None, 
    controls=None, 
    chroms=None, 
	in_window=2114,
    out_window=1000, 
    max_jitter=128, 
    min_counts=None,
	max_counts=None, 
    verbose=False
):
	"""Extract sequences and signals at coordinates from a locus file.
	This function will take in genome-wide sequences, signals, and optionally
	controls, and extract the values of each at the coordinates specified in
	the locus file/s and return them as tensors.
	Signals and controls are both lists with the length of the list, n_s
	and n_c respectively, being the middle dimension of the returned
	tensors. Specifically, the returned tensors of size 
	(len(loci), n_s/n_c, (out_window/in_wndow)+max_jitter*2).
	The values for sequences, signals, and controls, can either be filepaths
	or dictionaries of np arrays or a mix of the two. When a filepath is 
	passed in it is loaded using pyfaidx or pyBigWig respectively.   
	Parameters
	----------
	loci: str or pd.DataFrame or list/tuple of such
		Either the path to a bed file or a pd DataFrame object containing
		three columns: the chromosome, the start, and the end, of each locus
		to train on. Alternatively, a list or tuple of strings/DataFrames where
		the intention is to train on the interleaved concatenation, i.e., when
		you want to train on peaks and negatives.
	sequences: str or dictionary
		Either the path to a fasta file to read from or a dictionary where the
		keys are the unique set of chromosoms and the values are one-hot
		encoded sequences as np arrays or memory maps.
	signals: list of strs or list of dictionaries or None, optional
		A list of filepaths to bigwig files, where each filepath will be read
		using pyBigWig, or a list of dictionaries where the keys are the same
		set of unique chromosomes and the values are np arrays or memory
		maps. If None, no signal tensor is returned. Default is None.
	controls: list of strs or list of dictionaries or None, optional
		A list of filepaths to bigwig files, where each filepath will be read
		using pyBigWig, or a list of dictionaries where the keys are the same
		set of unique chromosomes and the values are np arrays or memory
		maps. If None, no control tensor is returned. Default is None. 
	chroms: list or None, optional
		A set of chromosomes to extact loci from. Loci in other chromosomes
		in the locus file are ignored. If None, all loci are used. Default is
		None.
	in_window: int, optional
		The input window size. Default is 2114.
	out_window: int, optional
		The output window size. Default is 1000.
	max_jitter: int, optional
		The maximum amount of jitter to add, in either direction, to the
		midpoints that are passed in. Default is 128.
	min_counts: float or None, optional
		The minimum number of counts, summed across the length of each example
		and across all tasks, needed to be kept. If None, no minimum. Default 
		is None.
	max_counts: float or None, optional
		The maximum number of counts, summed across the length of each example
		and across all tasks, needed to be kept. If None, no maximum. Default 
		is None.  
	verbose: bool, optional
		Whether to display a progress bar while loading. Default is False.
	Returns
	-------
	seqs: torch.tensor, shape=(n, 4, in_window+2*max_jitter)
		The extracted sequences in the same order as the loci in the locus
		file after optional filtering by chromosome.
	signals: torch.tensor, shape=(n, len(signals), out_window+2*max_jitter)
		The extracted signals where the first dimension is in the same order
		as loci in the locus file after optional filtering by chromosome and
		the second dimension is in the same order as the list of signal files.
		If no signal files are given, this is not returned.
	controls: torch.tensor, shape=(n, len(controls), out_window+2*max_jitter)
		The extracted controls where the first dimension is in the same order
		as loci in the locus file after optional filtering by chromosome and
		the second dimension is in the same order as the list of control files.
		If no control files are given, this is not returned.
	"""

	seqs, signals_, controls_ = [], [], []
	in_width, out_width = in_window // 2, out_window // 2

	# Load the sequences
	if isinstance(sequences, str):
		sequences = pyfaidx.Fasta(sequences)

	names = ['chrom', 'start', 'end']
	if not isinstance(loci, (tuple, list)):
		loci = [loci]

	loci_dfs = []
	for i, df in enumerate(loci):
		if isinstance(df, str):
			df = pd.read_csv(df, sep='\t', usecols=[0, 1, 2], header=None, index_col=False, names=names)
			df['idx'] = np.arange(len(df)) * len(loci) + i
		loci_dfs.append(df)

	loci = pd.concat(loci_dfs).set_index("idx").sort_index().reset_index(drop=True)
	if chroms is not None:
		loci = loci[np.isin(loci['chrom'], chroms)]

	# Load the signal and optional control tracks if filenames are given
	if signals is not None:
		for i, signal in enumerate(signals):
			if isinstance(signal, str):
				signals[i] = pyBigWig.open(signal, "r")

	if controls is not None:
		for i, control in enumerate(controls):
			if isinstance(control, str):
				controls[i] = pyBigWig.open(control, "r")

	desc = "Loading Loci"
	d = not verbose

	max_width = max(in_width, out_width)

	for chrom, start, end in tqdm(loci.values, disable=d, desc=desc):
		mid = start + (end - start) // 2

		if start - max_width - max_jitter < 0:
			continue

		if end + max_width + max_jitter >= len(sequences[chrom]):
			continue
		
		start = mid - out_width - max_jitter
		end = mid + out_width + max_jitter
		
		# Extract the signal from each of the signal files
		if signals is not None:
			signals_.append([])
			for signal in signals:
				if isinstance(signal, dict):
					signal_ = signal[chrom][start:end]
				else:
					signal_ = signal.values(chrom, start, end, numpy=True)
					signal_ = np.nan_to_num(signal_)

				signals_[-1].append(signal_)

		# For the sequences and controls extract a window the size of the input
		start = mid - in_width - max_jitter
		end = mid + in_width + max_jitter

		# Extract the controls from each of the control files
		if controls is not None:
			controls_.append([])
			for control in controls:
				if isinstance(control, dict):
					control_ = control[chrom][start:end]
				else:
					control_ = control.values(chrom, start, end, numpy=True)
					control_ = np.nan_to_num(control_)

				controls_[-1].append(control_)

		# Extract the sequence
		if isinstance(sequences, dict):
			seq = sequences[chrom][start:end].T
		else:
			seq = eu.pp.ohe_seq(sequences[chrom][start:end].seq.upper())
		
		seqs.append(seq)

	seqs = torch.tensor(np.array(seqs), dtype=torch.float32)

	if signals is not None:
		signals_ = torch.tensor(np.array(signals_), dtype=torch.float32)

		idxs = torch.ones(signals_.shape[0], dtype=torch.bool)
		if max_counts is not None:
			idxs = (idxs) & (signals_.sum(dim=(1, 2)) < max_counts)
		if min_counts is not None:
			idxs = (idxs) & (signals_.sum(dim=(1, 2)) > min_counts)

		if controls is not None:
			controls_ = torch.tensor(np.array(controls_), dtype=torch.float32)
			return seqs[idxs], signals_[idxs], controls_[idxs]

		return seqs[idxs], signals_[idxs]
	else:
		if controls is not None:
			controls_ = torch.tensor(np.array(controls_), dtype=torch.float32)
			return seqs, controls_

		return seqs

In [53]:
import os

data_dir = "/cellar/users/aklie/data/eugene/avsec21/ENCSR000EGM/data"
reference_dir = "/cellar/users/aklie/data/eugene/avsec21/reference"
peaks = os.path.join(data_dir, "peaks.bed")
#peaks = "/cellar/users/aklie/data/eugene/avsec21/ENCSR000EGM/toy.bed"
seqs = os.path.join(reference_dir, "hg38.fa")
signals = [os.path.join(data_dir, "plus.bw"), os.path.join(data_dir, "minus.bw")]
controls = [os.path.join(data_dir, "control_plus.bw"), os.path.join(data_dir, "control_minus.bw")]

training_chroms = ['chr{}'.format(i) for i in range(1, 17)]
valid_chroms = ['chr{}'.format(i) for i in range(18, 23)]

In [7]:
X_toy, y_toy, X_ctl_toy = extract_loci(peaks, seqs, signals, controls, max_jitter=0)

In [8]:
X_toy.shape, y_toy.shape, X_ctl_toy.shape

(torch.Size([100, 4, 2114]),
 torch.Size([100, 2, 1000]),
 torch.Size([100, 2, 2114]))

In [9]:
from torch.utils.data import Dataset, DataLoader
class ProfileDataset(Dataset):
	"""A data generator for BPNet inputs.
	This generator takes in an extracted set of sequences, output signals,
	and control signals, and will return a single element with random
	jitter and reverse-complement augmentation applied. Jitter is implemented
	efficiently by taking in data that is wider than the in/out windows by
	two times the maximum jitter and windows are extracted from that.
	Essentially, if an input window is 1000 and the maximum jitter is 128, one
	would pass in data with a length of 1256 and a length 1000 window would be
	extracted starting between position 0 and 256. This  generator must be 
	wrapped by a PyTorch generator object.
	Parameters
	----------
	sequences: torch.tensor, shape=(n, 4, in_window+2*max_jitter)
		A one-hot encoded tensor of `n` example sequences, each of input 
		length `in_window`. See description above for connection with jitter.
	signals: torch.tensor, shape=(n, t, out_window+2*max_jitter)
		The signals to predict, usually counts, for `n` examples with
		`t` output tasks (usually 2 if stranded, 1 otherwise), each of 
		output length `out_window`. See description above for connection 
		with jitter.
	controls: torch.tensor, shape=(n, t, out_window+2*max_jitter) or None, optional
		The control signal to take as input, usually counts, for `n`
		examples with `t` strands and output length `out_window`. If
		None, does not return controls.
	in_window: int, optional
		The input window size. Default is 2114.
	out_window: int, optional
		The output window size. Default is 1000.
	max_jitter: int, optional
		The maximum amount of jitter to add, in either direction, to the
		midpoints that are passed in. Default is 0.
	reverse_complement: bool, optional
		Whether to reverse complement-augment half of the data. Default is False.
	random_state: int or None, optional
		Whether to use a deterministic seed or not.
	"""

	def __init__(
		self, 
		sequences, 
		signals, 
		controls=None, 
		in_window=2114, 
		out_window=1000, 
		max_jitter=0, 
		reverse_complement=False, 
		random_state=None
	):
		self.in_window = in_window
		self.out_window = out_window
		self.max_jitter = max_jitter
		
		self.reverse_complement = reverse_complement
		self.random_state = np.random.RandomState(random_state)

		self.signals = signals
		self.controls = controls
		self.sequences = sequences	

	def __len__(self):
		return len(self.sequences)

	def __getitem__(self, idx):
		#i = self.random_state.choice(len(self.sequences))
		j = 0 if self.max_jitter == 0 else self.random_state.randint(self.max_jitter*2) 

		X = self.sequences[idx][:, j:j+self.in_window]
		y = self.signals[idx][:, j:j+self.out_window]

		if self.controls is not None:
			X_ctl = self.controls[idx][:, j:j+self.in_window]

		if self.reverse_complement and self.random_state.choice(2) == 1:
			X = torch.flip(X, [0, 1])
			y = torch.flip(y, [0, 1])

			if self.controls is not None:
				X_ctl = torch.flip(X_ctl, [0, 1])

		if self.controls is not None:
			return X, X_ctl, y

		return X, y
	
	def to_dataloader(
		self, 
        batch_size=None, 
        pin_memory=True, 
        shuffle=True, 
        num_workers=0, 
        **kwargs
    ):
		"""Convert the dataset to a PyTorch DataLoader

		Parameters:
		----------
		batch_size (int, optional):
			batch size for dataloader
		pin_memory (bool, optional):
			whether to pin memory for dataloader
		shuffle (bool, optional):
			whether to shuffle the dataset
		num_workers (int, optional):
			number of workers for dataloader
		**kwargs:
			additional arguments to pass to DataLoader
		"""
		batch_size = batch_size if batch_size is not None else eu.settings.batch_size
		return DataLoader(
			self,
			batch_size=batch_size,
			pin_memory=pin_memory,
			shuffle=shuffle,
			num_workers=num_workers,
			**kwargs
		)

In [10]:
toy_dataset = ProfileDataset(
    X_toy,
    y_toy,
    X_ctl_toy,
)

In [11]:
toy_dataset[0][0].shape, toy_dataset[0][1].shape, toy_dataset[0][2].shape

(torch.Size([4, 2114]), torch.Size([2, 2114]), torch.Size([2, 1000]))

In [12]:
toy_dataloader = toy_dataset.to_dataloader(shuffle=False)

In [13]:
batch = next(iter(toy_dataloader))
batch[0].shape, batch[1].shape, batch[2].shape

(torch.Size([100, 4, 2114]),
 torch.Size([100, 2, 2114]),
 torch.Size([100, 2, 1000]))

In [14]:
batch[0]

tensor([[[1., 1., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 1.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 1.],
         [0., 1., 0.,  ..., 0., 1., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.]],

        [[1., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 0., 0

In [15]:
from bpnetlite.io import PeakGenerator

In [16]:
bpnetlite_dataloader = PeakGenerator(
    peaks,
    seqs,
    signals,
    controls,
    max_jitter=0,
    batch_size=128,
    random_state=13
)   

In [17]:
bpnetlite_batch = next(iter(bpnetlite_dataloader))
bpnetlite_batch[0].shape, bpnetlite_batch[1].shape, bpnetlite_batch[2].shape

(torch.Size([100, 4, 2114]),
 torch.Size([100, 2, 2114]),
 torch.Size([100, 2, 1000]))

In [18]:
bpnetlite_batch[0]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 1.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

# BPNet model

In [19]:
# bpnetlite
import torch
torch.backends.cudnn.benchmark = True

class BPNet(torch.nn.Module):
	"""A basic BPNet model with stranded profile and total count prediction.
	This is a reference implementation for BPNet. The model takes in
	one-hot encoded sequence, runs it through: 
	(1) a single wide convolution operation 
	THEN 
	(2) a user-defined number of dilated residual convolutions
	THEN
	(3a) profile predictions done using a very wide convolution layer 
	that also takes in stranded control tracks 
	AND
	(3b) total count prediction done using an average pooling on the output
	from 2 followed by concatenation with the log1p of the sum of the
	stranded control tracks and then run through a dense layer.
	This implementation differs from the original BPNet implementation in
	two ways:
	(1) The model concatenates stranded control tracks for profile
	prediction as opposed to adding the two strands together and also then
	smoothing that track 
	(2) The control input for the count prediction task is the log1p of
	the strand-wise sum of the control tracks, as opposed to the raw
	counts themselves.
	(3) A single log softmax is applied across both strands such that
	the logsumexp of both strands together is 0. Put another way, the
	two strands are concatenated together, a log softmax is applied,
	and the MNLL loss is calculated on the concatenation. 
	(4) The count prediction task is predicting the total counts across
	both strands. The counts are then distributed across strands according
	to the single log softmax from 3.
	Parameters
	----------
	n_filters: int, optional
		The number of filters to use per convolution. Default is 64.
	n_layers: int, optional
		The number of dilated residual layers to include in the model.
		Default is 8.
	n_outputs: int, optional
		The number of profile outputs from the model. Generally either 1 or 2 
		depending on if the data is unstranded or stranded. Default is 2.
	alpha: float, optional
		The weight to put on the count loss.
	name: str or None, optional
		The name to save the model to during training.
	trimming: int or None, optional
		The amount to trim from both sides of the input window to get the
		output window. This value is removed from both sides, so the total
		number of positions removed is 2*trimming.
	verbose: bool, optional
		Whether to display statistics during training. Setting this to False
		will still save the file at the end, but does not print anything to
		screen during training. Default is True.
	"""

	def __init__(self, n_filters=64, n_layers=8, n_outputs=2, 
		n_control_tracks=2, alpha=1, profile_output_bias=True, 
		count_output_bias=True, name=None, trimming=None, verbose=True):
		super(BPNet, self).__init__()
		self.n_filters = n_filters
		self.n_layers = n_layers
		self.n_outputs = n_outputs
		self.n_control_tracks = n_control_tracks

		self.alpha = alpha
		self.name = name or "bpnet.{}.{}".format(n_filters, n_layers)
		self.trimming = trimming or 2 ** n_layers

		self.iconv = torch.nn.Conv1d(4, n_filters, kernel_size=21, padding=10)
		self.irelu = torch.nn.ReLU()

		self.rconvs = torch.nn.ModuleList([
			torch.nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=2**i, 
				dilation=2**i) for i in range(1, self.n_layers+1)
		])
		self.rrelus = torch.nn.ModuleList([
			torch.nn.ReLU() for i in range(1, self.n_layers+1)
		])

		self.fconv = torch.nn.Conv1d(n_filters+n_control_tracks, n_outputs, 
			kernel_size=75, padding=37, bias=profile_output_bias)
		
		n_count_control = 1 if n_control_tracks > 0 else 0
		self.linear = torch.nn.Linear(n_filters+n_count_control, 1, 
			bias=count_output_bias)

	def forward(self, X, X_ctl=None):
		"""A forward pass of the model.
		This method takes in a nucleotide sequence X, a corresponding
		per-position value from a control track, and a per-locus value
		from the control track and makes predictions for the profile 
		and for the counts. This per-locus value is usually the
		log(sum(X_ctl_profile)+1) when the control is an experimental
		read track but can also be the output from another model.
		Parameters
		----------
		X: torch.tensor, shape=(batch_size, 4, sequence_length)
			The one-hot encoded batch of sequences.
		X_ctl: torch.tensor, shape=(batch_size, n_strands, sequence_length)
			A value representing the signal of the control at each position in 
			the sequence.
		Returns
		-------
		y_profile: torch.tensor, shape=(batch_size, n_strands, out_length)
			The output predictions for each strand.
		"""
		start, end = self.trimming, X.shape[2] - self.trimming

		X = self.irelu(self.iconv(X))
		for i in range(self.n_layers):
			X_conv = self.rrelus[i](self.rconvs[i](X))
			X = torch.add(X, X_conv)

		if X_ctl is None:
			X_w_ctl = X
		else:
			X_w_ctl = torch.cat([X, X_ctl], dim=1)

		y_profile = self.fconv(X_w_ctl)[:, :, start:end]

		# counts prediction
		X = torch.mean(X[:, :, start-37:end+37], dim=2)

		if X_ctl is not None:
			X_ctl = torch.sum(X_ctl[:, :, start-37:end+37], dim=(1, 2))
			X_ctl = X_ctl.unsqueeze(-1)
			X = torch.cat([X, torch.log(X_ctl+1)], dim=-1)

		y_counts = self.linear(X).reshape(X.shape[0], 1)
		return y_profile, y_counts

In [20]:
import torchinfo

In [21]:
model = BPNet(n_outputs=2, n_control_tracks=2, trimming=(2114 - 1000) // 2).cuda()

In [22]:
torchinfo.summary(model, input_size=((32, 4, 2114), (32, 2, 2114)))

Layer (type:depth-idx)                   Output Shape              Param #
BPNet                                    [32, 2, 1000]             --
├─Conv1d: 1-1                            [32, 64, 2114]            5,440
├─ReLU: 1-2                              [32, 64, 2114]            --
├─ModuleList: 1-17                       --                        (recursive)
│    └─Conv1d: 2-1                       [32, 64, 2114]            12,352
├─ModuleList: 1-18                       --                        --
│    └─ReLU: 2-2                         [32, 64, 2114]            --
├─ModuleList: 1-17                       --                        (recursive)
│    └─Conv1d: 2-3                       [32, 64, 2114]            12,352
├─ModuleList: 1-18                       --                        --
│    └─ReLU: 2-4                         [32, 64, 2114]            --
├─ModuleList: 1-17                       --                        (recursive)
│    └─Conv1d: 2-5                       [32, 6

In [23]:
outs = model(batch[0].cuda(), batch[1].cuda())

In [24]:
outs[0].shape, outs[1].shape

(torch.Size([100, 2, 1000]), torch.Size([100, 1]))

# Training

In [36]:
# autoreload
%load_ext autoreload
%autoreload 2

In [54]:
X_toy_train, y_toy_train, X_ctl_toy_train = eu.dl.read_profile(peaks, seqs, signals, controls, max_jitter=128, chroms=training_chroms)
X_toy_val, y_toy_val, X_ctl_toy_val = eu.dl.read_profile(peaks, seqs, signals, controls, max_jitter=0, chroms=valid_chroms)

In [55]:
X_toy_train.shape, y_toy_train.shape, X_ctl_toy_train.shape

(torch.Size([45803, 4, 2370]),
 torch.Size([45803, 2, 1256]),
 torch.Size([45803, 2, 2370]))

In [56]:
X_toy_val.shape, y_toy_val.shape, X_ctl_toy_val.shape

(torch.Size([7051, 4, 2114]),
 torch.Size([7051, 2, 1000]),
 torch.Size([7051, 2, 2114]))

In [57]:
from eugene.dataload import ProfileDataset

In [ ]:
X_toy_dataset_train = ProfileDataset(
    X_toy_train,
    y_toy_train,
    X_ctl_toy_train
)
X_toy_dataset_val = ProfileDataset(
    X_toy_val,
    y_toy_val,
    X_ctl_toy_val
)

In [46]:
X_toy_dataloader_train = X_toy_dataset_train.to_dataloader(batch_size=64, workers=4, shuffle=True)
X_toy_dataloader_val = X_toy_dataset_val.to_dataloader(batch_size=64, workers=4, shuffle=False)

In [ ]:
from eugene.models.base._ProfileModel import ProfileModel
from eugene.models._profile_models import BPNet

model = BPNet(
    input_len=2114,
    output_dim=1000,
    n_outputs=2,
    n_control_tracks=2, 
    trimming=(2114 - 1000) // 2
).cuda()

In [ ]:
outs = model(batch[0].cuda(), batch[1].cuda())
outs[0].shape, outs[1].shape

In [47]:
import pytorch_lightning as pl

In [48]:
trainer = pl.Trainer(gpus=1, max_epochs=1, logger=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [52]:
trainer.fit(model, X_toy_dataloader_train, X_toy_dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | iconv  | Conv1d     | 5.4 K 
1 | irelu  | ReLU       | 0     
2 | rconvs | ModuleList | 98.8 K
3 | rrelus | ModuleList | 0     
4 | fconv  | Conv1d     | 9.9 K 
5 | linear | Linear     | 66    
--------------------------------------
114 K     Trainable params
0         Non-trainable params
114 K     Total params
0.457     Total estimated model params size (MB)


/cellar/users/aklie/opt/miniconda3/envs/eugene_dev/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 13


/cellar/users/aklie/opt/miniconda3/envs/eugene_dev/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


# Attributions

In [256]:
import seqexplainer

In [257]:
seqexplainer.attribute?

Signature:
seqexplainer.attribute(
    model,
    inputs: torch.Tensor,
    method: Union[str, Callable],
    target: int = 0,
    device: str = 'cpu',
    **kwargs,
)
Docstring: <no docstring>
File:      ~/projects/ML4GLand/SeqExplainer/seqexplainer/_feature_attribution.py
Type:      function


In [267]:
seqexplainer.attribute(
    pretrained_model,
    X,
    method="InputXGradient",
    additional_forward_args=(X_ctl,),
    target=0
)

AttributeError: 'tuple' object has no attribute 'shape'